In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!git clone https://github.com/BillyGun27/license_plate_YOLO

Cloning into 'license_plate_YOLO'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 76 (delta 16), reused 76 (delta 16), pack-reused 0
Unpacking objects: 100% (76/76), done.


In [3]:
%cd license_plate_YOLO

/content/license_plate_YOLO


In [4]:
!cp "/content/gdrive/My Drive/Dataset Plat Mobil/VOCdevkit.zip" .
!unzip VOCdevkit.zip

Archive:  VOCdevkit.zip
   creating: VOCdevkit/VOCplat/
   creating: VOCdevkit/VOCplat/Annotations/
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_101916.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_101939.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102016.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102030.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102116.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102146.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102149.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102156.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102201.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102245.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102337.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102338.xml  
  inflating: VOCdevkit/VOCplat/Annotations/IMG_20190704_102341.xml  
  i

In [0]:
!python voc_custom_annotation.py

In [6]:
"""
Retrain the YOLO model for your own dataset.
"""

import keras.backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping 
from keras.layers import Input, Lambda

from utils.core import test_yolo_loss as yolo_loss
from utils.utils  import get_random_data
from utils.train_tool import get_classes,get_anchors,data_generator_wrapper
from utils.evaluation import AveragePrecision
import numpy as np

import argparse

#change model here
#from model.yolo3 import yolo_body
from model.small_mobilenets2 import yolo_body
#from model.mobilenet import yolo_body
#from model.medium_darknet import yolo_body
#from model.yolo3 import tiny_yolo_body


def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/yolo_weights.h5'):
    '''create the training model'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))
   
    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
   
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (87, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))
 
    for y in range(-3, 0):
        model_body.layers[y].name = "conv2d_output_" + str(h//{-3:32, -2:16, -1:8}[y])

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def create_tiny_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2,
            weights_path='model_data/tiny_yolo_weights.h5'):
    '''create the training model, for Tiny YOLOv3'''
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16}[l], w//{0:32, 1:16}[l], \
        num_anchors//2, num_classes+5)) for l in range(2)]

    model_body = tiny_yolo_body(image_input, num_anchors//2, num_classes)
    print('Create Tiny YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze the darknet body or freeze all but 2 output layers.
            num = (20, len(model_body.layers)-2)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    for y in range(-2, 0):
        model_body.layers[y].name = "conv2d_output_" + str(h//{-2:32, -1:16}[y])

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.7})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model


Using TensorFlow backend.


In [7]:
epoch_end_first = 50#75
epoch_end_final = 100#200
model_name = 'plat_yolo'
log_dir = 'logs/000/'
model_path = ''

train_path = 'plat.txt'
val_path = 'plat_test.txt'
classes_path = 'class/plat_classes.txt'
anchors_path = 'anchors/plat_yolo_anchors.txt'
    
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416,416) # multiple of 32, hw
num_anchors = len(anchors)
image_input = Input(shape=(None, None, 3))

is_tiny_version = len(anchors)==6 # default setting
if is_tiny_version:
    model = create_tiny_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
else:
    model = create_model(input_shape, anchors, num_classes, load_pretrained=False,
        freeze_body=2, weights_path=model_path) # make sure you know what you freeze

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True,  period=1)#save_best_only=True,
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

with open(train_path) as f:
        train_lines = f.readlines()

with open(val_path) as f:
    val_lines = f.readlines()

# with open(test_path) as f:
#     test_lines = f.readlines()

num_train = int(len(train_lines))
num_val = int(len(val_lines))

'''
val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)

num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

train_lines = lines[:num_train]
val_lines = lines[num_train:]

num_train = int(len(train_lines))
num_val = int(len(val_lines))
'''

print('Train on {} samples, val on {} samples.'.format(num_train, num_val))

W0710 07:15:47.190835 140355301611392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0710 07:15:47.235364 140355301611392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0710 07:15:47.244627 140355301611392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0710 07:15:47.245794 140355301611392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

/usr/

17227776/17225924 [==============================] - 3s 0us/step


W0710 07:15:58.567260 140355301611392 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0710 07:15:59.150717 140355301611392 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3080: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Create YOLOv3 model with 9 anchors and 1 classes.
Train on 80 samples, val on 5 samples.


In [9]:
# Train with frozen layers first, to get a stable loss.
    # Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
    if True:
        model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
             'yolo_loss' : lambda y_true, y_pred: y_pred})

        batch_size = 32#24#32

        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        history = model.fit_generator(data_generator_wrapper(train_lines, batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, num_train//batch_size),
                validation_data=data_generator_wrapper(val_lines, batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, num_val//batch_size),
                epochs=epoch_end_first,
                initial_epoch=0,
                callbacks=[logging, checkpoint])#, meanAP

      
        last_loss = history.history['loss'][-1]
        last_val_loss = history.history['val_loss'][-1]

        hist = "loss{0:.4f}-val_loss{1:.4f}".format(last_loss,last_val_loss)

        model.save_weights(log_dir + "last_"+ hist + ".h5")

        model.save_weights(log_dir + model_name+'_trained_weights_stage_1.h5')

Train on 80 samples, val on 5 samples, with batch size 32.
Epoch 1/50
2/2 [==============================] - 47s 23s/step - loss: 8455.5872 - val_loss: 7547.1147
Epoch 2/50
2/2 [==============================] - 14s 7s/step - loss: 7193.1401 - val_loss: 5853.8706
Epoch 3/50
2/2 [==============================] - 42s 21s/step - loss: 5882.5344 - val_loss: 5320.1406
Epoch 4/50
2/2 [==============================] - 41s 21s/step - loss: 5067.0549 - val_loss: 4951.0483
Epoch 5/50
2/2 [==============================] - 41s 21s/step - loss: 4406.9373 - val_loss: 4716.4946
Epoch 6/50
2/2 [==============================] - 41s 21s/step - loss: 3741.2228 - val_loss: 4831.8799
Epoch 7/50
2/2 [==============================] - 41s 21s/step - loss: 3239.4781 - val_loss: 4489.5464
Epoch 8/50
2/2 [==============================] - 42s 21s/step - loss: 2975.2008 - val_loss: 4084.0000
Epoch 9/50
2/2 [==============================] - 42s 21s/step - loss: 2584.8684 - val_loss: 3997.6243
Epoch 10/50
2/2

In [10]:
# Unfreeze and continue training, to fine-tune.
    # Train longer if the result is not good.
    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(lr=1e-4), loss={ 
            'yolo_loss' : lambda y_true, y_pred: y_pred}) # recompile to apply the change
        print('Unfreeze all of the layers.')

        batch_size =  32#24#32 note that more GPU memory is required after unfreezing the body


        print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
        history = model.fit_generator(data_generator_wrapper(train_lines, batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(val_lines, batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=epoch_end_final,
            initial_epoch=epoch_end_first,
            callbacks=[logging, checkpoint, reduce_lr ])#, meanAP, early_stopping

        last_loss = history.history['loss'][-1]
        last_val_loss = history.history['val_loss'][-1]

        hist = "loss{0:.4f}-val_loss{0:.4f}".format(last_loss,last_val_loss)

        model.save_weights(log_dir + "last_"+ hist + ".h5")

        model.save_weights(log_dir + model_name + '_trained_weights_final.h5')

    # Further training if needed.

Unfreeze all of the layers.
Train on 80 samples, val on 5 samples, with batch size 32.
Epoch 51/100
2/2 [==============================] - 50s 25s/step - loss: 292.6120 - val_loss: 376.4022
Epoch 52/100
2/2 [==============================] - 40s 20s/step - loss: 317.6608 - val_loss: 381.7111
Epoch 53/100
2/2 [==============================] - 41s 20s/step - loss: 286.0351 - val_loss: 369.3742
Epoch 54/100
2/2 [==============================] - 42s 21s/step - loss: 294.9616 - val_loss: 358.7036
Epoch 55/100
2/2 [==============================] - 41s 21s/step - loss: 282.7352 - val_loss: 348.5676
Epoch 56/100
2/2 [==============================] - 41s 21s/step - loss: 279.4407 - val_loss: 335.0514
Epoch 57/100
2/2 [==============================] - 41s 21s/step - loss: 271.3736 - val_loss: 339.3003
Epoch 58/100
2/2 [==============================] - 41s 21s/step - loss: 260.9942 - val_loss: 339.0999
Epoch 59/100
2/2 [==============================] - 42s 21s/step - loss: 270.9815 - val_l

In [16]:
"""
Retrain the YOLO model for your own dataset.
"""

import numpy as np
from keras.layers import Input,Reshape
from keras.models import Model,load_model

from utils.core import preprocess_true_boxes, yolo_loss
from utils.utils  import get_random_data
from utils.train_tool import get_classes,get_anchors
from utils.evaluate_tool import sigmoid, data_generator_wrapper_eval,numpy_box_iou,compute_ap
from tqdm import tqdm

#from model.yolo3 import tiny_yolo_body as yolo_body 

#from model.yolo3 import yolo_body
#from model.mobilenet import yolo_body
from model.small_mobilenets2 import yolo_body
#from model.squeezenet import yolo_body
#from model.medium_darknet import yolo_body


import argparse

def _main():
    
    weights_path = 'logs/000/plat_yolo_trained_weights_final.h5'
    
    test_path = 'plat.txt'
    #log_dir = 'logs/logits_only_000/'
    classes_path = 'class/plat_classes.txt'
    anchors_path = 'anchors/plat_yolo_anchors.txt'
    
    class_names = get_classes(classes_path)
    anchors = get_anchors(anchors_path)

    num_classes = len(class_names)
    num_anchors = len(anchors) #9

    shape_size = 416
    input_shape = (shape_size, shape_size) # multiple of 32, hw

    num_layers = num_anchors//3 #9//3

    #with open(train_path) as f:
    #    train_lines = f.readlines()
    #train_lines = train_lines[:200]

    #with open(val_path) as f:
    #    val_lines = f.readlines()
    #val_lines = val_lines[:150]

    with open(test_path) as f:
        test_lines = f.readlines()
    #test_lines = test_lines[:2]


    #num_train = int(len(train_lines))
    #num_val = int(len(val_lines))
    num_test = int(len(test_lines))

     #declare model
   
    image_input = Input(shape=(shape_size, shape_size, 3))
    
    try:
            eval_model = load_model(model_path, compile=False)
    except:
            eval_model = yolo_body(image_input, num_anchors//3, num_classes)#9//3
            eval_model.load_weights(weights_path)

    
    yolo_out = []
    fmap = shape_size//32
    mapsize = [1,2,4]

    if num_layers==3 :
        ly_out = [-3, -2, -1] 
    elif num_layers==2 :
        ly_out = [-2, -1] 
    else :
        ly_out = [-1] 

    # return the constructed network architecture
    # class+5
    for ly in range(num_layers):
        yolo_layer = Reshape(( fmap*mapsize[ly], fmap*mapsize[ly] , 3, (num_classes + 5) ))(eval_model.layers[ ly_out[ly] ].output)

        yolo_out.append(yolo_layer)
    
    eval_model = Model( inputs= eval_model.input , outputs= yolo_out )
    eval_model._make_predict_function()

    batch_size = 1
   
    all_detections  = [ [] for i in range(num_classes) ]
    all_annotations = [ [] for i in range(num_classes) ]

    count_detections  = [ [0 for i in range(num_classes)] for i in range(num_layers) ]
    total_object = 0

    datagen = data_generator_wrapper_eval(test_lines, batch_size, input_shape, anchors, num_classes,eval_model)
    
    
    print( "{} test data".format(num_test) )
    for n in tqdm( range(num_test) ):#num_test
        img,flogits,mlogits = next(datagen)

        for l in range(num_layers):
            #print( "layer" + str(l) )
            arrp = flogits[l]
            box = np.where(arrp[...,4] > 0 )
            box = np.transpose(box)

            for i in range(len(box)):
                #print("obj" + str(i) )
                #detection_label =  np.argmax( flogits[l][tuple(box[i])][5:]) 
                annotation_label =  np.argmax( flogits[l][tuple(box[i])][5:]) 

                #print( "{} ({}) {} == ({}) {} ".format(l, detection_label, class_names[  detection_label ] ,annotation_label, class_names[  annotation_label ] ) )
                
                all_detections[annotation_label].append( mlogits[l][tuple(box[i])] ) 
                all_annotations[annotation_label].append( flogits[l][tuple(box[i])] )

                count_detections[l][annotation_label] +=1
                total_object +=1
    

    print(len(all_detections) )
    print(len(all_annotations) )
    print(count_detections)
    print(total_object)

    
    conf_thres = 0.2#0.5
    iou_thres = 0.45
    
    average_precisions = {}

    for label in tqdm( range( num_classes ) ) : 
        
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))

        
        num_detect = len( all_detections[label] )
        for det in  range( num_detect ):

            detect_box = all_detections[label][det][...,0:4]
            detect_conf = all_detections[label][det][...,4]
            detect_label =  np.argmax( all_detections[label][det][...,5:] ) 

            annot_box = all_annotations[label][det][...,0:4]
            annot_conf = all_annotations[label][det][...,4]
            detect_label =  np.argmax( all_detections[label][det][...,5:] ) 
            
            iou = numpy_box_iou( detect_box , annot_box)

            scores = np.append(scores, detect_conf )

        
            if( iou > iou_thres and  detect_conf > conf_thres and (label == detect_label ) ):
                #print( best_iou[tuple(box[i])] )
                #print("pos")
                false_positives = np.append(false_positives, 0)
                true_positives   = np.append(true_positives, 1)
            else:
                #print("neg")
                false_positives = np.append(false_positives, 1)
                true_positives  = np.append(true_positives, 0)
                
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]
        #print(true_positives)

        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)
        #print(true_positives)

        recall = true_positives  / num_detect
        #print( recall )
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        #print( precision )

        average_precision  = compute_ap(recall, precision)
        average_precisions[label] = average_precision
    
    print( "loaded weights {}".format(weights_path) )

    #print(average_precisions)

    for label, average_precision in average_precisions.items():
        print(class_names[label] + ': {:.4f}'.format(average_precision))
    print('mAP: {:.4f}'.format(sum(average_precisions.values()) / len(average_precisions)))   
    
  

if __name__ == '__main__':
    _main()

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
  0%|          | 0/80 [00:00<?, ?it/s]

80 test data
/content/license_plate_YOLO/VOCdevkit/VOCplat/JPEGImages/IMG_20190704_102337.jpg 1113,2448,2389,2886,0



  2%|▎         | 2/80 [00:03<02:51,  2.20s/it]

/content/license_plate_YOLO/VOCdevkit/VOCplat/JPEGImages/IMG_20190704_102358.jpg 1832,2291,2808,2648,0



  4%|▍         | 3/80 [00:03<02:06,  1.64s/it]

...

/content/license_plate_YOLO/VOCdevkit/VOCplat/JPEGImages/IMG_20190704_105203.JPG 1753,2200,2910,2595,0



 99%|█████████▉| 79/80 [00:30<00:00,  2.82it/s]

/content/license_plate_YOLO/VOCdevkit/VOCplat/JPEGImages/IMG_20190704_114322.jpg 1746,1686,2589,2043,0



100%|██████████| 1/1 [00:00<00:00, 168.07it/s]

/content/license_plate_YOLO/VOCdevkit/VOCplat/JPEGImages/IMG_20190704_112751.JPG 1675,2029,2613,2381,0

1
1
[[0], [0], [80]]
80
loaded weights logs/000/plat_yolo_trained_weights_final.h5
plat: 0.4095
mAP: 0.4095
